In [1]:
#! git clone --branch feature/Tomas https://github.com/prerakmody/CS4180-DL

In [2]:
! nvidia-smi

/bin/sh: 1: nvidia-smi: not found


In [3]:
#encoding:utf-8
import sys
import math
import os

dir_main = './CS4180-DL'
sys.path.append(dir_main)

import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
from torchvision import models
import torch.nn.functional as F

from src.pruning.weightPruning.layers import MaskedLinear,MaskedConv2d

cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

def getYOLOv1(name=''):
    if name != '':
        myYOLO            = YOLOv1(name, cfg['D'], batch_norm=True)
        VGG               = models.vgg16_bn(pretrained=True)
        state_dict_VGG    = VGG.state_dict()
        state_dict_myYOLO = myYOLO.state_dict()
        
        for k in state_dict_VGG.keys():
            if k in state_dict_myYOLO.keys() and k.startswith('features'):
                state_dict_myYOLO[k] = state_dict_VGG[k]
        myYOLO.load_state_dict(state_dict_myYOLO)
        return myYOLO

    else:
        print (' - Pass a name for your model')
        sys.exit(1)
    


class YOLOv1(nn.Module):

    def __init__(self, name, cfg, batch_norm, image_size=448):
        super(YOLOv1, self).__init__()
        self.name       = name
        self.features   = self.getFeatureLayers(cfg, batch_norm)
        self.linear1    = MaskedLinear(512 * 7 * 7, 4096)
        self.linear2    = MaskedLinear(4096, 1470)
        self.classifier = nn.Sequential( # add the regression part to the features
            # nn.Linear(512 * 7 * 7, 4096),
            self.linear1,
            nn.ReLU(True),
            nn.Dropout(),
            # nn.Linear(4096, 1470),
            self.linear2,
        )
        self._initialize_weights()
        self.image_size = image_size

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        x = torch.sigmoid(x)
        x = x.view(-1,7,7,30)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


    def getFeatureLayers(self, cfg, batch_norm=False):
        if (1):
            params_in_channels  = 3
            params_conv_stride  = 1
            params_conv_size    = 3 
            params_first_flag   = True
            params_pool_stride  = 2
            params_pool_kernel  = 2

        layers = []
        for item in cfg:
            params_conv_stride = 1
            if (item == 64 and params_first_flag):
                params_conv_stride = 2
                params_first_flag  = False

            if item == 'M': # max-pooling
                layers += [nn.MaxPool2d(kernel_size=params_pool_kernel, stride=params_pool_stride)]
            else:
                params_kernels = item
                conv2d = MaskedConv2d(params_in_channels, params_kernels, kernel_size=params_conv_size, stride=params_conv_stride, padding=1)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(item), nn.ReLU(inplace=True)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=True)]
                params_in_channels = item
        return nn.Sequential(*layers)

    def set_masks(self, masks):
        count = 0
        for feat in self.features:
          try:
            if feat.name == 'MaskedConv2d':
              feat.set_mask(masks[count])
              count += 1
          except:
            pass
        self.linear1.set_mask(masks[count])
        self.linear2.set_mask(masks[count + 1])

def test():
    net = getYOLOv1()
    img = torch.rand(1,3,448,448)
    img = Variable(img)
    output = net(img)
    print(output.size())

# if __name__ == '__main__':
#     test()

In [4]:
import os
import sys
import pdb; # pdb.set_trace()
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tensorboardcolab import TensorBoardColab

USE_GPU = torch.cuda.is_available()
print (' - USE_GPU : ', USE_GPU)

 - USE_GPU :  False


Using TensorFlow backend.


In [5]:
LOGGER = TensorBoardColab()    

Wait for 8 seconds...
TensorBoard link:
http://cd20ed12.ngrok.io


In [6]:
print('Hello!')
print(sys.path)

from src.dataloader import YoloDataset
# from src.nets import *
from src.train import YOLOv1Train, YOLOv1Loss
# from src.train import YOLOv1Loss

torch.cuda.empty_cache()

print('Hello!')

if __name__ == "__main__":
    
    if (1):
        model_name = 'yolov1'
        model = getYOLOv1(model_name)
        print (' - 1. Net : ', model_name)
    
    if (1):
        DEBUG         = True
        LEARNING_RATE = 0.001
        EPOCHS        = 8
        BATCH_SIZE    = 16
        criterion     = YOLOv1Loss('','',5,0.5)
        optimizer     = 'SGD'
        
        print (' - 2. [PARAMS] Debug      : ', DEBUG)
        print (' - 2. [PARAMS] BATCH_SIZE : ', BATCH_SIZE)
        print (' - 2. [PARAMS] EPOCHS     : ', EPOCHS)
        print (' - 2. [PARAMS] Optimizer  : ', optimizer)
        
    if (1):
        CHKP_DIR    = 'chkpoints'
        CHKP_NAME   = 'yolov1_epoch%.3d.pkl' % (4)
        CHKP_EPOCHS = 1
        CHKP_LOAD = True
        print (' - 2. [PARAMS_SAVE] CHKP_DIR  : ', CHKP_DIR)
        print (' - 2. [PARAMS_SAVE] CHKP_NAME : ', CHKP_NAME)
        
    if (1):
        dir_annotations  = os.path.join(dir_main, 'data/VOCdevkit_trainval/VOC2007')
        file_annotations = os.path.join(dir_main,'data/VOCdevkit_trainval/VOC2007/anno_trainval.txt')
        image_size       = 448
        grid_num         = 7
        flag_augm        = 0
        trainFlag        = True

        YoloDatasetTrain = YoloDataset(dir_annotations, file_annotations
                                    , trainFlag
                                    , image_size, grid_num
                                    , flag_augm
                                    , transform = [transforms.ToTensor()] )
        DataLoaderTrain = DataLoader(YoloDatasetTrain, batch_size=BATCH_SIZE, shuffle=False,num_workers=0)
        
        dir_annotations  = os.path.join(dir_main, 'data/VOCdevkit_test/VOC2007')
        file_annotations = os.path.join(dir_main, 'data/VOCdevkit_test/VOC2007/anno_test.txt')
        trainFlag        = False
        YoloDatasetTest  = YoloDataset(dir_annotations, file_annotations
                                    , trainFlag
                                    , image_size, grid_num
                                    , flag_augm
                                    , transform = [transforms.ToTensor()] )
        DataLoaderTest   = DataLoader(YoloDatasetTest, batch_size=BATCH_SIZE, shuffle=False,num_workers=0)
        
        print(' - 3. [TrainDataset] %d images' % (len(YoloDatasetTrain)))
    
    if (1):
        print (' - 4. Logger ')
        pass
#         LOGGER = TensorBoardColab()
    
    if (1):
        print (' - 5. Training')
        print (' -------------------------------------------------------------------- ')
        trainObj = YOLOv1Train()
        trainObj.train(model, criterion, optimizer
                            , DataLoaderTrain, DataLoaderTest
                            , LEARNING_RATE, EPOCHS, BATCH_SIZE
                            , USE_GPU, LOGGER
                            , CHKP_LOAD, CHKP_DIR, CHKP_NAME, CHKP_EPOCHS
                            , DEBUG)
        
  

Hello!
['/home/tomas/Documents/TUDelft/DeepLearning/CS4180-DL/demo/demo2', '/home/tomas/dohop/repo', '/home/tomas/dohop/repo/metabooking/exec_bots', '/home/tomas/dohop/repo/metabooking/booking-essentials', '/home/tomas/dohop/repo/robots', '/home/tomas/dohop/repo/py-dohop-geonames/src', '/home/tomas/Documents/TUDelft/virtualenvs/delft/lib/python37.zip', '/home/tomas/Documents/TUDelft/virtualenvs/delft/lib/python3.7', '/home/tomas/Documents/TUDelft/virtualenvs/delft/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7', '', '/home/tomas/Documents/TUDelft/virtualenvs/delft/lib/python3.7/site-packages/datasets-0.0.9-py3.7.egg', '/home/tomas/Documents/TUDelft/virtualenvs/delft/lib/python3.7/site-packages', '/home/tomas/Documents/TUDelft/virtualenvs/delft/src/lasagne', '/home/tomas/Documents/TUDelft/virtualenvs/delft/lib/python3.7/site-packages/IPython/extensions', '/home/tomas/.ipython', '/home/tomas/Documents/TUDelft/DeepLearning/CS4180-DL']
Hello!
 - 1. Net :  yolov1
 - 2. [PARAMS] Debug

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [ ]:
# trainObj.model.parameters
pruning_perc = 90.
masks = weight_prune(trainObj.model, pruning_perc)
trainObj.model.set_masks(masks)

In [ ]:
prune_rate(trainObj.model)

In [ ]:
trainObj.train(trainObj.model, criterion, optimizer
                            , DataLoaderTrain, DataLoaderTest
                            , LEARNING_RATE, EPOCHS, BATCH_SIZE
                            , USE_GPU, LOGGER
                            , CHKP_LOAD, CHKP_DIR, CHKP_NAME, CHKP_EPOCHS
                            , DEBUG)